In [1]:
import glob
import pandas as pd

data_files = glob.glob("chess_ratings/*.csv")
all_dfs = [pd.read_csv(file, sep='|') for file in data_files]

/tmp/ipykernel_2329/750563276.py:5: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  all_dfs = [pd.read_csv(file, sep='|') for file in data_files]


In [2]:
data = pd.concat(all_dfs)
del all_dfs
data["Year / month"] = [f"{y} / {m if (len(str(m)) == 2) else ('0' + str(m))}" for y, m in zip(data["Year"], data["Mon"])]

In [55]:
data

,ID Number,Name,Fed,Sex,Tit,WTit,OTit,FOA,Rat,Gms,K,B-day,Flag,Year,Mon,Year / month
0,644498,,FRA,M,,,,,2008,1,20,0,,2017,12,2017 / 12
1,20504578,,MAW,M,,,,,1617,0,40,0,,2017,12,2017 / 12
2,35077023,A Chakravarthy,IND,M,,,,,1151,0,40,1986,i,2017,12,2017 / 12
3,10207538,"A E M, Doshtagir",BAN,M,,,,,1840,0,40,1974,i,2017,12,2017 / 12
4,10206612,"A K M, Sourab",BAN,M,,,,,1708,4,40,0,,2017,12,2017 / 12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352221,21836060,"Zyto, Karol",POL,M,,,,,1071,0,40,2006,,2019,12,2019 / 12
352222,34190632,"Zyubin, Sergey",RUS,M,,,,,1884,0,40,1975,i,2019,12,2019 / 12
352223,24232602,"Zyuzev, Pavel",RUS,M,,,,,1852,0,40,1993,i,2019,12,2019 / 12
352224,1189980,"Zywert, Kacper",POL,M,,,,,1565,0,20,1999,i,2019,12,2019 / 12


In [56]:
# unify flag columns
data.loc[data['Flag'] == 'wi', 'Flag'] = 'i'
data.loc[data['Flag'] == 'w', 'Flag'] = ''

In [57]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [5]:
data.isna().any(axis=0)

ID Number       False
Name             True
Fed             False
Sex             False
Tit              True
WTit             True
OTit             True
FOA              True
Rat             False
Gms             False
K               False
B-day           False
Flag             True
Year            False
Mon             False
Year / month    False
dtype: bool

In [6]:
string_cols = ['Name', 'Tit', 'WTit', 'OTit', 'FOA', 'Flag']

data[string_cols] = data[string_cols].fillna('')

In [84]:
smol = data.iloc[200000:300000]

df = spark.createDataFrame(smol)
df

DataFrame[ID Number: bigint, Name: string, Fed: string, Sex: string, Tit: string, WTit: string, OTit: string, FOA: string, Rat: bigint, Gms: bigint, K: bigint, B-day: bigint, Flag: string, Year: bigint, Mon: bigint, Year / month: string]

In [85]:
df.show()

+---------+--------------------+---+---+---+----+----+---+----+---+---+-----+----+----+---+------------+
|ID Number|                Name|Fed|Sex|Tit|WTit|OTit|FOA| Rat|Gms|  K|B-day|Flag|Year|Mon|Year / month|
+---------+--------------------+---+---+---+----+----+---+----+---+---+-----+----+----+---+------------+
| 14182017|       Petrov, Timur|UKR|  M|   |    |    |   |1444|  0| 40| 2007|    |2017| 12|   2017 / 12|
| 14529645|    Petrov, Tomislav|CRO|  M|   |    |    |   |1942|  0| 40| 1974|   i|2017| 12|   2017 / 12|
|  2901846|    Petrov, Valentin|FID|  M|   |    |    |   |2235|  0| 20| 1958|   i|2017| 12|   2017 / 12|
| 24141968|    Petrov, Valentin|RUS|  M|   |    |    |   |2114|  0| 20| 1983|   i|2017| 12|   2017 / 12|
|  2910691|      Petrov, Valeri|FID|  M|   |    |    |   |1940|  0| 20| 1954|   i|2017| 12|   2017 / 12|
|  2935244|     Petrov, Valeriy|FID|  M|   |    |    |   |1661|  0| 20| 1964|    |2017| 12|   2017 / 12|
|  4147952|      Petrov, Valery|RUS|  M|   |    |    | 

In [ ]:
from pyspark.sql.functions import mean, desc

df.filter(df["country"] == "france") \ # only french stations
  .groupBy("station_id") \ # by station
  .agg(mean("temperature").alias("average_temp")) \ # calculate average
  .orderBy(desc("average_temp")) \ # order by average 
  .take(100) # return first 100 rows

df.rdd \
  .filter(lambda x: x[1] == "france") \ # only french stations
  .map(lambda x: (x[0], x[2])) \ # select station & temp
  .mapValues(lambda x: (x, 1)) \ # generate count
  .reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1])) \ # calculate sum & count
  .mapValues(lambda x: x[0]/x[1]) \ # calculate average
  .sortBy(lambda x: x[1], ascending = False) \ # sort
  .take(100)

In [90]:
import numpy as np


def reduce_by_months(df, what, agg_func, agg_func_name='', n_months=1):
    idmap = {c: i for i, c in enumerate(df.columns)}
    name = f"{what}:{agg_func_name}"
        
    if n_months > 12:
        raise ValueError("Invalid months: ", n_months)
    elif n_months == 12:
        key_func = lambda x: (x[idmap['ID Number']], x[idmap['Year']])
        out_key_func = lambda x: (x[0][0], f"{x[0][1]}_yearly", x[1])
    else:
        key_func = lambda x: (x[idmap['ID Number']], x[idmap['Year']], x[idmap['Mon']] // n_months)
        out_key_func = lambda x: (x[0][0], f"{x[0][1]}/{int(x[0][2] * n_months)}_{n_months}", x[1])
    
    res = df.rdd \
            .map(lambda x: (key_func(x), x[idmap[what]])) \
            .reduceByKey(lambda x: np.mean(x)) \
            .map(out_key_func) \
            .toDF(['ID Number', 'Colname', name])
    
    return res

In [91]:
red = reduce_by_months(df, 'Rat', np.mean, agg_func_name='mean', n_months=3) \
        .groupBy('ID Number') \
        .pivot('Colname') \
        .max('Rat:mean')

red.show(5)

+---------+---------+--------+
|ID Number|2017/12_3|2018/0_3|
+---------+---------+--------+
|   351210|     1843|    null|
|   615978|     1701|    null|
|  4694546|     1861|    null|
| 21814538|     1321|    null|
|   776866|     1586|    null|
+---------+---------+--------+
only showing top 5 rows

